In [1]:
import pandas as pd
import numpy as np

In [2]:
akas_url = ('https://datasets.imdbws.com/title.akas.tsv.gz')
basics_url =('https://datasets.imdbws.com/title.basics.tsv.gz')
rating_url= ('https://datasets.imdbws.com/title.ratings.tsv.gz')

In [3]:
basics = pd.read_csv(basics_url, sep ='\t', low_memory =False)
akas = pd.read_csv(akas_url, sep ='\t', low_memory =False)
rating = pd.read_csv(rating_url, sep ='\t', low_memory =False)

In [4]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
rating.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1839
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


# Data Cleaning

In [7]:
#replace "\N " with np.nan
basics = basics.replace({'\\N':np.nan})

In [8]:
#Elimintae movies that are null for runtimeMinutes column
basics = basics.dropna(subset = 'runtimeMinutes')

In [9]:
#Elimintae movies that are null for genre
basics = basics.dropna(subset = 'genres')

In [10]:
#Keep only titleType == Movie
basics = basics.loc[basics['titleType'] == 'movie']

In [11]:
#keep startyear 2000-2022
#errors='coerce' to handle any invalid values
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce')
basics = basics.loc[(basics['startYear'] >= 2000) & (basics['startYear']<=2022)]

In [12]:
#Eliminate movies that include 'Documentary'
is_documentaries = basics['genres'].str.contains('documentary',case = False)
basics = basics[~is_documentaries]

In [13]:
#replace "\N" with np.nan in akas table
akas = akas.replace({"\\N":np.nan})

In [14]:
#replace "\N" with np.nan in rating table
rating = rating.replace({"\\N":np.nan})

In [15]:
#keep only U.S movie in akas
akas = akas.loc[akas['region'] == 'US']

In [16]:
#filter the basics table down to only include the US by using the filter akas
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9963353     True
9963362     True
9963401    False
9963446     True
9963530    False
Name: tconst, Length: 147812, dtype: bool

In [17]:
#filter the basics with keepers
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9962818,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9963213,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9963353,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9963362,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [18]:
keepers = rating['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1324468    False
1324469    False
1324470    False
1324471    False
1324472    False
Name: tconst, Length: 1324473, dtype: bool

In [19]:
rating = rating[keepers]
rating

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2625
5,tt0000006,5.1,182
6,tt0000007,5.4,820
...,...,...,...
1324434,tt9916200,8.1,230
1324435,tt9916204,8.2,264
1324442,tt9916348,8.3,18
1324443,tt9916362,6.4,5390


# Saving to data folder

In [20]:
import os
os.makedirs('Data/', exist_ok=True)
os.listdir ("Data/")

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_rating.csv.gz']

In [21]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [22]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [23]:
rating.to_csv("Data/title_rating.csv.gz",compression='gzip',index=False)

In [24]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1447772 entries, 5 to 36380713
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1447772 non-null  object
 1   ordering         1447772 non-null  int64 
 2   title            1447772 non-null  object
 3   region           1447772 non-null  object
 4   language         3971 non-null     object
 5   types            980613 non-null   object
 6   attributes       46867 non-null    object
 7   isOriginalTitle  1446430 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.4+ MB


In [25]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86847 entries, 34803 to 9963446
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86847 non-null  object 
 1   titleType       86847 non-null  object 
 2   primaryTitle    86847 non-null  object 
 3   originalTitle   86847 non-null  object 
 4   isAdult         86847 non-null  object 
 5   startYear       86847 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86847 non-null  object 
 8   genres          86847 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


In [26]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502224 entries, 0 to 1324448
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502224 non-null  object 
 1   averageRating  502224 non-null  float64
 2   numVotes       502224 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB
